In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pprint import pprint

test_URL = "https://fastestlaps.com/models/sauber-c5-lm"

urlreq = requests.get(test_URL).text

soup = BeautifulSoup(urlreq)

soup.title

<title>Sauber C5 LM specs, lap times, performance data - FastestLaps.com</title>

In [2]:
colsm6 = soup.find_all("div", class_ = "col-sm-6")

colsm6col0 = colsm6[0].find_all("div", class_ = "section")

In [3]:
listtitles = []
listvalues = []
toggle = True

itemtable = colsm6col0[0].find("table")

for item in itemtable.find_all("tr"):
    for subitem in item.find_all("td"):
        if toggle:
            listtitles.append(subitem.text)
        else:
            listvalues.append(subitem.text)
        toggle = not(toggle)

listtitles

# We would like to use 'Car Type', 'Origin County'
# For Year, there is 'Years built' and 'Introduced'. We will use the first year posted

['Car type',
 'Curb weight',
 'Years built',
 'Origin country',
 'Views',
 'Submitted by']

In [4]:
colsm6col1 = colsm6[1]

for item in colsm6col1.find_all("div", class_ = "section"):
    targetitem = item.find("h3", id= "powertrainSpecs")
    if targetitem is not None:
        powertrainItem = item
        break

In [5]:
powertitles = []
powervalues = []
toggle = True

for poweritems in powertrainItem.find_all("tr"):
    for subitem in poweritems.find_all("td"):
        if toggle:
            powertitles.append(subitem.text)
        else:
            powervalues.append(subitem.text)
        toggle = not(toggle)

powertitles

['Engine type', 'Displacement', 'Transmission', 'Layout']

In [6]:
additionaldict = {
    "Car_type":[],
    "Year":[],
    "Country_of_Origin": [],
    "Engine_type":[],
    "Displacement": [],
    "Transmission": [],
    "Engine_Layout": [],
    "Drivetrain": []
}

In [7]:
# Car Type
additionaldict["Car_type"].append(listvalues[listtitles.index("Car type")].strip())

# Year
if "Introduced" in listtitles:
    additionaldict["Year"].append(int(listvalues[listtitles.index("Introduced")]))
elif "Years built" in listtitles:
    additionaldict["Year"].append(int(listvalues[listtitles.index("Years built")][0:4]))

# Country of Origin
additionaldict["Country_of_Origin"].append(listvalues[listtitles.index("Origin country")].strip())

# Engine type
additionaldict["Engine_type"].append(powervalues[powertitles.index("Engine type")].strip())

# Displacement (in liters)
disp = powervalues[powertitles.index("Displacement")].split(" ")[0]

additionaldict["Displacement"].append(float(disp))

# Transmission
additionaldict["Transmission"].append(powervalues[powertitles.index("Transmission")].strip())

# Splitting up the Layout
SplitLayout = powervalues[powertitles.index("Layout")].split(",")

additionaldict["Engine_Layout"].append(SplitLayout[0].strip())

additionaldict["Drivetrain"].append(SplitLayout[1].strip())

additionaldict

{'Car_type': ['Convertible'],
 'Year': [1978],
 'Country_of_Origin': ['Switzerland'],
 'Engine_type': ['BMW-Madern M12 2.0L S4'],
 'Displacement': [2.0],
 'Transmission': ['Hewland 5-speed Manual'],
 'Engine_Layout': ['middle engine'],
 'Drivetrain': ['rear wheel drive']}